In [11]:
import re
import string
import pandas as pd
from math import log
import os
import random
import hnswlib
import re
import string
from collections import Counter
from sklearn.cluster import KMeans

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")

import pickle
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

from transformers import AutoTokenizer, AutoModel
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/installer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
final_counts = []

In [3]:
def clean_text(text):
    text = str(text).lower().strip() 
    text = re.sub("(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*?)", "", text)
    text = re.sub('[^a-z\s]+', "", text)
    text = text.translate (str.maketrans('', '', string.punctuation))
      # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    # text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [4]:
test = r"with bscmath https://dasds.com please please 🙏🙏 🙏🙏 sir mujhe koi government jobs arrange kara dijiye sir please mai indian army me join hona chachta hu mai apne sabhi indian army ke maut ka badala loga sir please sir',' the prime minister of india dear sir this"

In [5]:
clean_text(test)

'with bscmath please please sir mujhe koi government jobs arrange kara dijiye sir please mai indian army me join hona chachta hu mai apne sabhi indian army ke maut ka badala loga sir please sir the prime minister of india dear sir this'

In [6]:
all_roman_words = Counter()

In [7]:
pref = "/home/installer/ps/long_term/data/"

In [8]:
for file in os.listdir("/home/installer/ps/long_term/data/"):
    if not file.endswith("csv"):
        continue

    file_path = os.path.join("/home/installer/ps/long_term/data/", file)
    print(file, "here")
    
    with open(file_path, 'r') as f:
        df = pd.read_csv(f, usecols=[0, 21], header=None, low_memory=False)
        df.columns = ['reg_no', 'subject_content']
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)

        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        for sent in df.subject_content_cleaned.values:
            res = re.findall(r'\w+', sent)
            all_roman_words.update(res)
        print(file, "Size of updated set", str(len(all_roman_words)))

    # print(file, "Size of updated set", str(len(all_roman_words)))
    


chunk3.csv here


KeyboardInterrupt: 

In [ ]:
import pickle

In [ ]:

with open('all_roman_words.pkl', 'wb') as f:
    pickle.dump(all_roman_words, f, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
len(all_roman_words)

3588204

In [ ]:
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [ ]:
max_len = max(len(w) for w in all_roman_words.keys())


In [ ]:
def Pword(word, N=sum(all_roman_words.values())): 
    "Probability of `word`."
    return all_roman_words[word] / N

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(Pword(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=max_len):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]
    
@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

The segment(text) function takes a string text as input and returns a list of words that is the most probable segmentation of the input text. The function uses a dynamic programming algorithm to calculate the probability of all possible segmentations of the input text and returns the segmentation with the highest probability.

Here's how the segment(text) function works:

The function first checks if the input text is an empty string. If it is, the function returns an empty list since there are no words to segment.

Otherwise, the function generates all possible splits of the input text into a first word and the rest of the text using the splits(text, start, L) function.
For each split, the function recursively calls itself on the rest of the text to generate a list of words for that split.

The function concatenates the first word with the list of words generated from the rest of the text for each split to form a list of candidate segmentations.

The function calculates the probability of each candidate segmentation using the Pwords(words) function, which assumes that each word is independent of others and calculates the product of the probability of each word in the segmentation.

The function returns the candidate segmentation with the highest probability using the max() function and the key argument to specify the probability function to use.

Overall, the segment(text) function is a recursive implementation of a dynamic programming algorithm called the Viterbi algorithm, which is commonly used in natural language processing tasks such as part-of-speech tagging and named entity recognition.

In [ ]:
def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

In [ ]:
segment('dearsirmadamwithveryrespectiwishtodrawyourattentionandforyounecessaryactiontorefundmyrefundableamountthatihasalreadyinformaboutmyproblemtorailwaydepartmentbymailatkolrrbatgmailcomonapratamandonereminderalsosendonapratpmcopyalsosendatrrbkolkatarecruitmentboardrailwayrecruitmentcellkolkatabuttillnownoreplaycometomenothinghastakenproblemisthatregardingrecruitmentlevelpostcentheresultsofcbtcenwasdeclaredonunfortunatelyididnotshortlistedforthenextstapephysicalefficiencytestpetaspermentionedinthecentralizedemploymentnoticecenifanycandidateappaireatexaminationcentereforststagebutnotqualifiedforndstagehisherfeerswillberefundedontheirbankaccountfromthiseveiwishtodrawyourattentionthattillnowididnotgetbackmyrefundableamountialsocheckonyourrrbkolkatawebsitetoupdatecorrectionmyaccountitshownthatyourrefundhasbeeninitiatedtothebankaccountdetailsgivenbyyouduringregistrationsothatirequestyoupleaselookintothismatterrefundtherefundableamountwhichmaybeusedforfreshcentralizedemploymentnoticenoteatthetimeofpaymentipayrsfivehundredonlyregistrationidyoursfaithfullythankingyouanupkumarmaitymyaccountdetailsnameanupkumarmaityacnoifsccodesbinnachindabranchbankstatebankofindia')


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
import sys
print(sys.setrecursionlimit(5000))

None


In [ ]:
def increase_count(words, k):
    result = []
    for word in words:
        result += [word] * k
    return result

In [ ]:
all_roman_words.most_common(10)

[('the', 29682649),
 ('to', 24699016),
 ('of', 24597148),
 ('and', 20535093),
 ('in', 14664176),
 ('i', 12515548),
 ('is', 11948551),
 ('for', 10168298),
 ('my', 9278816),
 ('a', 7969260)]

In [ ]:
for k in list(all_roman_words):
    try:
        tmp = segment(k)
        if (len(tmp)>1):
            tmp = increase_count(tmp, all_roman_words[k])
            all_roman_words.update(tmp)
            del all_roman_words[k]
    except:
        print(k)
    # del all_roman_words[k]
    

...


In [ ]:
all_roman_words.most_common(10)

[('the', 29781825),
 ('to', 24848649),
 ('of', 24654567),
 ('and', 20631035),
 ('in', 14934250),
 ('i', 13622733),
 ('is', 12101544),
 ('for', 10232339),
 ('my', 9334415),
 ('a', 8518635)]

In [ ]:
with open('all_roman_words_segmented.pkl', 'wb') as f:
    pickle.dump(all_roman_words, f, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
with open('/home/installer/ps/hnsw_rewording/all_roman_words_segmented.pkl', 'rb') as f:
    all_roman_words = pickle.load(f)

In [26]:
all_roman_words.most_common(10)

[('the', 29781825),
 ('to', 24848649),
 ('of', 24654567),
 ('and', 20631035),
 ('in', 14934250),
 ('i', 13622733),
 ('is', 12101544),
 ('for', 10232339),
 ('my', 9334415),
 ('a', 8518635)]

In [17]:
def fit_hnsw_index(features, ef=100, M=32):
    num_elements = len(features)
    labels_index = np.arange(num_elements)
    EMBEDDING_SIZE = len(features[0])
    
    p = hnswlib.Index(space='l2', dim=EMBEDDING_SIZE)# possible space options are l2, cosine or ip
    p.init_index(max_elements=num_elements, ef_construction=ef, M=M)
    int_labels = p.add_items(features, labels_index)
    p.set_ef(ef) 
    return p

In [18]:
def freq(sent):
    arr = np.zeros(26)
    for letter in sent:
        if 97 <= ord(letter) <= 122:
            # print("here")
            arr[ord(letter) - 97] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [ ]:
df = pd.DataFrame(all_roman_words.keys(), columns= ['word'])
df['histograms'] = df['word'].apply(freq)
df.reset_index(drop=True, inplace=True)
df

,word,histograms
0,my,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,grievance,"[0.30151134457776363, 0.0, 0.30151134457776363..."
2,i,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,am,"[0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,uploading,"[0.3333333333333333, 0.0, 0.0, 0.3333333333333..."
...,...,...
2781712,ipgmeampr,"[0.2773500981126146, 0.0, 0.0, 0.0, 0.27735009..."
2781713,indiscriminatoey,"[0.18257418583505536, 0.0, 0.18257418583505536..."
2781714,growfitter,"[0.0, 0.0, 0.0, 0.0, 0.2672612419124244, 0.267..."
2781715,gworfitter,"[0.0, 0.0, 0.0, 0.0, 0.2672612419124244, 0.267..."


In [ ]:
with open('df_histograms.pkl', 'wb') as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
with open('/home/installer/ps/hnsw_rewording/df_histograms.pkl', 'rb') as f:
    df = pickle.load(f)

In [15]:
K = 50 #nearest neighbours you wish to find for each grievance
l2_param = 0.001 #threshold for l2 distance
min_docs = 1 #minimum number of grievances in a set to call it a bulk

In [19]:
X = [item.tolist() for item in df['histograms'].values]

p = fit_hnsw_index(X, ef=K*10)
# ann_neighbor_indices, ann_distances = p.knn_query(X, K)

In [20]:
df

,word,histograms
0,my,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,grievance,"[0.30151134457776363, 0.0, 0.30151134457776363..."
2,i,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,am,"[0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,uploading,"[0.3333333333333333, 0.0, 0.0, 0.3333333333333..."
...,...,...
2781712,ipgmeampr,"[0.2773500981126146, 0.0, 0.0, 0.0, 0.27735009..."
2781713,indiscriminatoey,"[0.18257418583505536, 0.0, 0.18257418583505536..."
2781714,growfitter,"[0.0, 0.0, 0.0, 0.0, 0.2672612419124244, 0.267..."
2781715,gworfitter,"[0.0, 0.0, 0.0, 0.0, 0.2672612419124244, 0.267..."


In [21]:
def find_nearest(index, word, k=50):
    ann_neighbor_indices, ann_distances = index.knn_query(freq(word), k)
    out = np.array([df.loc[i, "word"].values for i in ann_neighbor_indices])
    ann_distances = np.array(ann_distances)
    return out



In [22]:
primary_dict = {w: find_nearest(p, w) for w in all_roman_words.keys()}


In [23]:
primary_dict

{}

In [ ]:
with open('primary_dict.pkl', 'wb') as f:
    pickle.dump(primary_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [83]:
# primary_dict = {} 

# for i in range(len(ann_neighbor_indices)):
#     indices = ann_neighbor_indices[i]
#     distances = ann_distances[i]
#     tmp = {}
#     for k,v in zip(indices, distances):
#         tmp[k] = v
#     primary_dict[i] = tmp

# # bulk_set = set()
# bulk_all = dict()

# # i = 1
# for id in primary_dict.keys():
#     tmp_docs = []
#     # if id in bulk_set:
#     #     continue
#     tmp_docs.append(df.loc[id, "word"])
#     # print(id)
#     for k,v in primary_dict[id].items():
#         # print(k,v)
#         if k == id: 
#             continue
#         if float(v) < l2_param:
#             tmp_docs.append(df.loc[k, "word"])
#             # bulk_set.add(k)

#     # print(tmp_docs)
#     # if len(tmp_docs)>min_docs:
#     bulk_all[df.loc[id, "word"]] = tmp_docs
#         # i+=1

In [84]:
# bulk_all

{'my': ['my', 'ym'],
 'grievance': ['grievance',
  'greivance',
  'griveance',
  'greviance',
  'gravience',
  'grievanec',
  'grienvace',
  'girevance',
  'grivenace',
  'gerviance',
  'grievnace',
  'giervance',
  'grivanece',
  'grevaince',
  'grievacne',
  'griavence',
  'giverance'],
 'i': ['i', 'iii', 'iiiii', 'iiiiii', 'iiiiiiii', 'iiiiiii'],
 'am': ['am', 'ma', 'mama', 'maam', 'amma', 'aamm', 'mamama', 'amamamam'],
 'uploading': ['uploading', 'uplaoding'],
 'in': ['in', 'ni'],
 'pdf': ['pdf', 'dfp', 'pfd', 'fdp', 'fpd'],
 'format': ['format'],
 'the': ['the', 'teh', 'het', 'hte', 'eht'],
 'government': ['government',
  'governemnt',
  'govenrmnet',
  'governemtn',
  'governmnet',
  'goevrnment',
  'govenrment',
  'governmetn',
  'govermnent',
  'govrenment',
  'gorvenment',
  'gonverment'],
 'did': ['did'],
 'not': ['not', 'otn'],
 'get': ['get', 'tge', 'teg', 'gte', 'etg'],
 'any': ['any',
  'nay',
  'annyay',
  'nya',
  'nayyan',
  'yan',
  'ayn',
  'nnayay',
  'nayayn'],
 'f

In [ ]:
def known(words, lwords): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in lwords)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def candidates(word, lwords): 
    "Generate possible spelling corrections for word."
    return( known(edits1(word), lwords) or known(edits2(word), lwords) or known(word, lwords) or [word])


def P(word, N=sum(all_roman_words.values())): 
    "Probability of `word`."
    return all_roman_words[word] / N

def correction(word, lwords): 
    "Most probable spelling correction for word."
    return max(candidates(word, lwords), key=P)




In [ ]:
tmp = dict()
for key in primary_dict.keys():
    tmp[key] = candidates(key, primary_dict[key])

In [ ]:
tmp

{'my': {'my', 'ym'},
 'grievance': {'girevance',
  'greivance',
  'grievacne',
  'grievance',
  'grievanec',
  'grievnace',
  'griveance'},
 'i': {'i'},
 'am': {'am', 'ma'},
 'uploading': {'uplaoding', 'uploading'},
 'in': {'in', 'ni'},
 'pdf': {'pdf', 'pfd'},
 'format': {'format'},
 'the': {'hte', 'teh', 'the'},
 'government': {'goevrnment',
  'govenrment',
  'govermnent',
  'governemnt',
  'government',
  'governmetn',
  'governmnet',
  'govrenment'},
 'did': {'did'},
 'not': {'not'},
 'get': {'get', 'gte'},
 'any': {'any', 'ayn', 'nay'},
 'facilities': {'faciliteis', 'facilities', 'faciltiies'},
 'please': {'lpease', 'pelase', 'plaese', 'pleaes', 'please', 'plesae'},
 'read': {'raed', 'read'},
 'this': {'this', 'thsi', 'tihs'},
 'matter': {'matter'},
 'banna': {'banan', 'banna', 'bnana'},
 'sarkar': {'sakrar', 'sarakr', 'sarkar', 'srakar'},
 'mo': {'mo', 'om'},
 'no': {'no', 'on'},
 'kcc': {'ckc', 'kcc'},
 'dear': {'daer', 'dear', 'dera', 'edar'},
 'pm': {'mp', 'pm'},
 'sir': {'sir'

In [34]:
# # WORDS = all_roman_words

# def P(word, N=sum(WORDS.values())): 
#     "Probability of `word`."
#     return WORDS[word] / N

# def correction(word): 
#     "Most probable spelling correction for word."
#     return max(candidates(word), key=P)



# def known(words): 
#     "The subset of `words` that appear in the dictionary of WORDS."
#     return set(w for w in words if w in WORDS)

# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'abcdefghijklmnopqrstuvwxyz'
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     return set(deletes + transposes + replaces + inserts)

# def edits2(word): 
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in edits1(word) for e2 in edits1(e1))


# def candidates(word): 
#     "Generate possible spelling corrections for word."
#     return( known(edits1(word)) or known(edits2(word)) or known(word) or [word])

# def tokens(text):
#     "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
#     return re.findall('[a-z]+', text.lower()) 

In [35]:
english_words = set()
with open("word.list") as f:
    for line in f.readlines():
        line = line.strip()
        english_words.add(line)

In [36]:
letters    = 'abcdefghijklmnopqrstuvwxyz'
for l in letters:
    if l=='a':
        continue
    try:
        # del WORDS[l]
        del bulk_all[l]
    except:
        print(l, "doesn't exist")

In [37]:
# def correction1(word):
#     w = correction(word)
#     print(w)
#     while P(w)<P(correction(w)):
#         w = correction(w)
#         print(w)
#     return w

In [38]:
# all_roman_words['cocanut']

In [39]:
# bulk_all['cocanut']

In [40]:
# known(candidates("bkna"))

In [41]:
# known(candidates("may"))

In [42]:
# for w in ['may', 'amy', 'yam', 'mya', 'mayyam', 'maymay', 'yma', 'aym']:
#     print(correction(w))


In [43]:
# "sampling" in english_words

In [44]:
# correction("sample")

In [45]:
# correction("knab")

TypeError: correction() missing 1 required positional argument: 'lwords'

In [ ]:
final_wording = dict()

In [ ]:
import pickle

# Open the file in binary mode
with open('trans_hindi.pickle', 'rb') as f:
    # Load the object from the file
    trans_hindi = pickle.load(f)

In [ ]:
trans_hindi = set(trans_hindi.keys())


In [ ]:
len(trans_hindi)

393946

In [ ]:
for key in bulk_all.keys():
    val = bulk_all[key]
    # word = find_word_with_minimum_hashes(val)
    for v in val:
        if v in english_words or v in trans_hindi:
            continue
        
        w = correction(v, val)
        while w != correction(v, val):
            w = correction(v, val)
        
        if w in final_wording:
            final_wording[v] = final_wording[w]
        else:
            final_wording[v] = w
        

In [ ]:
final_wording


{'ym': 'my',
 'mymy': 'mymy',
 'greivance': 'grievance',
 'griveance': 'grievance',
 'greviance': 'grievance',
 'gravience': 'gravience',
 'grievanec': 'grievance',
 'grienvace': 'grievance',
 'girevance': 'grievance',
 'grivenace': 'grievance',
 'gerviance': 'grievance',
 'grievnace': 'grievance',
 'giervance': 'grievance',
 'grivanece': 'grivanece',
 'grevaince': 'grievance',
 'grievancegrievance': 'grievancegrievance',
 'grievacne': 'grievance',
 'griavence': 'griavence',
 'giverance': 'giverance',
 'maam': 'maam',
 'amma': 'maam',
 'amam': 'maam',
 'aamm': 'maam',
 'mmaa': 'mama',
 'mamama': 'mamama',
 'amamamam': 'amamamam',
 'uplaoding': 'uploading',
 'ni': 'in',
 'inni': 'inin',
 'nini': 'inin',
 'inin': 'inin',
 'ininininininininininininininin': 'ininininininininininininininin',
 'pdf': 'pdf',
 'dfp': 'fdp',
 'dpf': 'pdf',
 'pfd': 'pdf',
 'fdp': 'fdp',
 'fpd': 'fdp',
 'fromat': 'format',
 'atform': 'atform',
 'teh': 'the',
 'hte': 'the',
 'thethe': 'thethe',
 'eththe': 'eththe'

In [ ]:
with open("fw.txt", "w") as f:
    for k, v in final_wording.items():
        f.write(k+" "+v+"\n")

In [ ]:
correction("desitnation")

'designation'

In [ ]:
candidates("desitnation")

{'designation', 'desigtnation', 'desination', 'desitnation', 'destination'}

In [ ]:
P("desitnation")

9.422650727524277e-09

In [ ]:
P("designation")

3.255525826359638e-05

In [ ]:
correction("indians")

'indian'

In [ ]:
"india" in english_words

False

## To Do
1. We need to try trie/tree approach (auto-correct). Read about it
2. Check the translation time complexity using m2m for gujarati and tamil
